- Angleblock의 자석이 두 겹 (Outer, Inner)으로 배치되어 LAG가 경량화 됨
- Offset과 경계의 개념을 도입하여 LAG와 자석이 직접적인 간섭이 일어나지 않을 수 있도록 함
- 자석을 넣는 직사각형의 각도는 연속된 두 자석을 넣는 직사각형의 중심을 이은 선분으로 함

**V2와 V3의 차이점** : V2는 **연속된 두 자석을 넣는 직사각형의 중심을 이은 선분** 기준으로 자석을 넣는 직사각형의 각도를 정했지만, V3는 그와 달리 **직사각형을 넣는 직사각형이 수차 회전 중심점 (B-CRA)를 바라보도록** 정했다.



## Preview : 자석 경량 Angleblock 제작 DXF, Bruteforce 최적화

### V2 Outer Magnet DXF
**distbtw = 0.5mm, BEST**  <img src="./IMG/outer0.5.png" width="300"/> **GIF** <img src="./IMG/V2_outer.gif" width="300"/>

### V2 Inner Magnet DXF
**distbtw = 0.6mm, BEST** <img src="./IMG/inner0.6.png" width="300"/> **GIF** <img src="./IMG/V2_inner.gif" width="300"/>


## V2와 V3의 간단 대조 GIF : 

## Inner : 
V2 : <img src="./IMG/V2_inner.gif" width="300"/> V3 : <img src="./IMG/V3_inner.gif" width="300"/>

In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
from rectangle import *

# 모든 각도는 라디안 단위로 표현됩니다.
# rectangle : (중심 x, 중심 y, 너비, 높이, 회전)

# 설계 정보
deltay = 0  # mm, LAG delta y
r0 = 60
r_AngleBlock = r0 + deltay

# 주축과 AngleBlock축 (편의상 이심축)의 기울기 관계
theta = 0  # AngleBlock 축에 대한 초기 각도

# 주축 기준 위치 계산
angleblock_theta = []  # 각도 (theta) 저장용 리스트
angleblock_x = []  # x 좌표 저장용 리스트
angleblock_y = []  # y 좌표 저장용 리스트
angleblock_ = []  # (theta, x, y) 튜플 저장용 리스트
angleblock_dot = []  # 점 (x, y) 저장용 리스트

CRA_point = (0, 0)  # CRA 중심점
magnet_dim = (10, 5)  # 자석의 크기 설정 (너비, 높이)
magnet_margin = 0.1  # 자석 여백 설정
maghole_dim = (10.3, 5.3)  # 자석 홀 크기 설정

offset_AB = 5
magnet_boundary_dim = (10, offset_AB * 2 + magnet_dim[1])
min_boundary = []  # 최소 경계 설정 (r, theta 인덱스 형식)
max_boundary = []  # 최대 경계 설정 (r, theta 인덱스 형식)

In [2]:
# CSV 파일에서 각도 블록 데이터를 불러오기
with open('AB_V3.csv', 'r') as file:
    for line in file:
        dot_ = list(map(float, line.strip().split(',')))
        print(dot_)
        angleblock_dot.append(dot_)
        angleblock_x.append(dot_[0])
        angleblock_y.append(dot_[1])

        angleblock_theta.append(math.atan2(dot_[1] - CRA_point[1], dot_[0] - CRA_point[0]))

# CRA 기준으로 최소 및 최대 경계 계산
for i in range(len(angleblock_dot)):
    min_boundary.append([distance_(angleblock_dot[i], CRA_point) - (magnet_boundary_dim[1] / 2), i])
    max_boundary.append([distance_(angleblock_dot[i], CRA_point) + (magnet_boundary_dim[1] / 2), i])

dot_cnt = len(angleblock_x)

# 주어진 위치와 크기 정보를 바탕으로 사각형을 생성하는 함수
def position_to_rect(pos, r_dim):  #pos : 원점 Angleblock 회전 축(0, 0) 기준 (r, theta). r_dim : (중심 x, 중심 y, 너비, 높이, 회전)
    angle = angleblock_theta[pos[1]]

    prevdot = angleblock_dot[pos[1] - 1]
    nextdot = angleblock_dot[(pos[1] + 1) % len(angleblock_theta)]

    rect_angle = math.atan2((nextdot[1] - prevdot[1]), (nextdot[0] - prevdot[0]))

    xcoord = pos[0] * math.cos(angle)
    ycoord = pos[0] * math.sin(angle)

    # CRA 기준 90도일 때 : [xcoord, ycoord, r_dim[0], r_dim[1], pos[1] - math.pi/2]
    return [xcoord, ycoord, r_dim[0], r_dim[1], rect_angle]

# 이진 탐색 함수 (binary search)
def binary_search(target, data):
    start = 0  # 맨 처음 위치
    end = len(data) - 1  # 맨 마지막 위치

    while start <= end:
        mid = (start + end) // 2  # 중간값 계산

        if data[mid] == target:
            return mid  # 타겟 위치 반환

        elif data[mid] > target:  # 타겟이 작으면 왼쪽을 더 탐색
            end = mid - 1
        else:  # 타겟이 크면 오른쪽을 더 탐색
            start = mid + 1
    return start, end

# 외부 자석 위치를 계산하는 함수
def outer_mag_position(distance_btw): 
    threshold = 0.1
    outer_mag_cnt = 1
    outer_mag_position = []  # r theta 형식으로 중심 위치 저장

    outer_mag_position.append([max_boundary[0][0] + magnet_boundary_dim[1] / 2, max_boundary[0][1]])

    for i in range(len(max_boundary)):

        r_ = max_boundary[i][0] + magnet_boundary_dim[1] / 2
        theta_ = max_boundary[i][1]
        nowrect = position_to_rect(outer_mag_position[outer_mag_cnt - 1], maghole_dim)
        nextrect = position_to_rect([r_, theta_], maghole_dim)

        firstrect = position_to_rect(outer_mag_position[0], maghole_dim)

        if min_distance_between_rectangles(nowrect, nextrect) < distance_btw:
            continue 
        elif min_distance_between_rectangles(firstrect, nextrect) > distance_btw * threshold: 
            outer_mag_position.append([r_, theta_])
            outer_mag_cnt += 1

    return outer_mag_position

# 자석 위치를 평가하는 함수 (r theta 점의 리스트를 입력으로 받음)
def evaluate_mag_position(mag_position): 
    r1 = position_to_rect(mag_position[0], maghole_dim)
    r2 = position_to_rect(mag_position[-1], maghole_dim)
    problem_dist = min_distance_between_rectangles(r1, r2)
    
    totaldist = 0
    size = len(mag_position)
    for i in range(size):
        r1 = position_to_rect(mag_position[i], maghole_dim)
        r2 = position_to_rect(mag_position[(i + 1) % size], maghole_dim)
        totaldist += min_distance_between_rectangles(r1, r2)
    reviseddistancebtw = totaldist / size  # 평균 자석 간 거리
    
    return problem_dist # 하지만 맨 처음에 배치하는 자석과 제일 마지막에 배치하게 되는 자석의 거리가 가장 컸기에 이를 반환하기로 결정

# 자석 위치를 DXF 파일로 저장하는 함수
def mag_pos_dxf(filename, mag_position): 
    from ezdxf import recover
    from ezdxf.addons.drawing import matplotlib
    import ezdxf

    doc = ezdxf.new()

    with open(filename, 'w') as f_: 
        pass
    msp = doc.modelspace()
    
    # 회전된 사각형을 생성하는 내부 함수
    def create_rotated_rectangle(rect):
        centerx, centery, width, height, angle = rect
        # 사각형의 꼭지점을 계산
        half_width = width / 2.0
        half_height = height / 2.0
        
        points = [
            (-half_width, -half_height),
            (half_width, -half_height),
            (half_width, half_height),
            (-half_width, half_height),
            (-half_width, -half_height)  # 루프를 닫기 위해 마지막 점을 다시 추가
        ]

        # 점들을 회전시키고 평행 이동
        rotated_points = []
        for x, y in points:
            # 회전
            xr = x * math.cos(angle) - y * math.sin(angle)
            yr = x * math.sin(angle) + y * math.cos(angle)
            # 평행 이동
            xt = xr + centerx
            yt = yr + centery
            rotated_points.append((xt, yt))

        # 회전된 점들로 사각형을 생성하여 모델 공간에 추가
        msp.add_lwpolyline(rotated_points)

    # DXF 파일을 표시하는 내부 함수
    def display_dxf(filename):
        # 예외 처리는 코드의 간결성을 위해 생략되었습니다.
        doc, auditor = recover.readfile(str(filename))
        if not auditor.has_errors:
            matplotlib.qsave(doc.modelspace(), str(filename[:-4]) + '.png')

    for pos in mag_position:
        create_rotated_rectangle(position_to_rect(pos, maghole_dim))

    doc.saveas(filename)
    display_dxf(filename)

# 내부 자석 위치를 계산하는 함수
def inner_mag_position(distance_btw): 
    threshold = 0.1
    inner_mag_position = []  # r theta 형식으로 중심 위치 저장

    # 내부 경계와의 교차 여부를 확인하는 내부 함수
    def rect_intersect_innerbound(rect): 

        dots = centerrectangle_to_dots(rect)

        thetaidxrange = [0 ,0]
        for dot in dots: 
            thetaofdot = math.atan2(dot[1], dot[0])

            d = binary_search(thetaofdot, angleblock_theta)
            thetaidxrange = [min(thetaidxrange[0], d[0], d[1]), max(thetaidxrange[1], d[1], d[0])]

        if (thetaidxrange[1] - thetaidxrange[0]) >= len(angleblock_theta) / 2: 
            thetaidxrange = [thetaidxrange[1], thetaidxrange[0] + len(angleblock_theta)]

        for idx in range(thetaidxrange[0], thetaidxrange[1]): 
            dots = min_boundary[idx]
            dot_x = dots[0] * math.cos(angleblock_theta[dots[1] % len(angleblock_theta)])
            dot_y = dots[0] * math.sin(angleblock_theta[dots[1] % len(angleblock_theta)])
            dotxy = [dot_x, dot_y]

            if point_inside_rect(rect, dotxy):
                return True
        return False

    # 첫 번째 자석 위치 배치
    step = 0.1
    
    pos = [min_boundary[0][0] - magnet_boundary_dim[1] / 2, 0]
    
    firstrect = position_to_rect(pos, maghole_dim)
    while rect_intersect_innerbound(firstrect):
        pos[0] -= step
        firstrect = position_to_rect(pos, maghole_dim)
    inner_mag_position.append(pos)

    for i in range(len(min_boundary)):
        pos = [min_boundary[i][0] - magnet_boundary_dim[1] / 2, i]
        while rect_intersect_innerbound(position_to_rect(pos, maghole_dim)):
            pos[0] -= step

        # 이전 자석과의 교차 확인
        nowrect = position_to_rect(inner_mag_position[-1], maghole_dim)
        nextrect = position_to_rect(pos, maghole_dim)

        if min_distance_between_rectangles(nowrect, nextrect) < distance_btw:
            continue
        elif min_distance_between_rectangles(firstrect, nextrect) > distance_btw * threshold: 
            inner_mag_position.append(pos)
    return inner_mag_position


[60.0, -25.0]
[59.998815651368226, -24.623011362066464]
[59.995262652228966, -24.246037606998843]
[59.98934114284854, -23.86909361707551]
[59.981051356997995, -23.49219427339975]
[59.9703936219439, -23.115354455312303]
[59.95736835843535, -22.738589039803927]
[59.94197608068748, -22.361912900928104]
[59.92421739636105, -21.985340909213825]
[59.9040930065385, -21.608887931078527]
[59.88160370569629, -21.2325688282412]
[59.856750381673514, -20.856398457135654]
[59.82953401563684, -20.48039166832404]
[59.79995568204179, -20.104563305910546]
[59.76801654859035, -19.72892820695541]
[59.7337178761848, -19.35350120088914]
[59.69706101887801, -18.978297108927105]
[59.65804742381994, -18.603330743484406]
[59.6166786312005, -18.2286169075911]
[59.57295627418882, -17.854170394307815]
[59.526882078868674, -17.480005986141745]
[59.478457864170394, -17.10613845446305]
[59.42768554179907, -16.732582558921713]
[59.37456711615906, -16.359353046864868]
[59.31910468427484, -15.986464652754576]
[59.261300

In [3]:
# 여러 거리 설정에 대해 자석 위치를 계산하고 DXF 파일로 저장
BTWrange = np.arange(0.1, 1, step=0.05)

for i in BTWrange:
    mag_pos = outer_mag_position(i)

    print("Evaluation of distbtw:", i, evaluate_mag_position(mag_pos)) # 가장 최적의 magnet 사이 거리 parameter를 grid-search 방식으로 찾음.

    mag_pos_dxf("outer" + str(i) + ".dxf", mag_pos)

Evaluation of distbtw: 0.1 4.96383349876715
Evaluation of distbtw: 0.15000000000000002 2.3532286280493238
Evaluation of distbtw: 0.20000000000000004 1.48124828055858
Evaluation of distbtw: 0.25000000000000006 0.6086055669208855
Evaluation of distbtw: 0.30000000000000004 0.6086055669208855
Evaluation of distbtw: 0.3500000000000001 8.858219249734978
Evaluation of distbtw: 0.40000000000000013 7.131193734257981
Evaluation of distbtw: 0.45000000000000007 3.224410435654627
Evaluation of distbtw: 0.5000000000000001 0.6086055669208855
Evaluation of distbtw: 0.5500000000000002 9.289041648552818
Evaluation of distbtw: 0.6000000000000002 7.995548398564313
Evaluation of distbtw: 0.6500000000000001 6.6984274875373915
Evaluation of distbtw: 0.7000000000000002 4.094657477022136
Evaluation of distbtw: 0.7500000000000002 3.659659317620264
Evaluation of distbtw: 0.8000000000000002 2.7889278625518235
Evaluation of distbtw: 0.8500000000000002 11.01186256375161
Evaluation of distbtw: 0.9000000000000002 7.5

## 결과
### Outer Magnet DXF
**distbtw = 0.5mm, BEST**
  
<img src="./IMG/outer0.5.png" alt="outer0.5.png" width="300"/>

**distbtw = 0.8mm, WORST**

<img src="./IMG/outer0.8.png" alt="outer0.8.png" width="300"/>

### Inner Magnet DXF
**distbtw = 0.6mm, BEST**
  
<img src="./IMG/inner0.6.png" alt="inner0.6.png" width="300"/>

**distbtw = 0.1mm, WORST**

<img src="./IMG/inner0.1.png" alt="inner0.1.png" width="300"/>

In [4]:
# 여러 거리 설정에 대해 자석 위치를 계산하고 DXF 파일로 저장
BTWrange = np.arange(0.1, 1, step=0.05)

for i in BTWrange:
    mag_pos = inner_mag_position(i)

    print("Evaluation of distbtw:", i, evaluate_mag_position(mag_pos)) # 가장 최적의 magnet 사이 거리 parameter를 grid-search 방식으로 찾음.

    mag_pos_dxf("inner" + str(i) + ".dxf", mag_pos)

Evaluation of distbtw: 0.1 10.164177959336426
Evaluation of distbtw: 0.15000000000000002 9.874786332094233
Evaluation of distbtw: 0.20000000000000004 9.006662192305772
Evaluation of distbtw: 0.25000000000000006 8.428078447519667
Evaluation of distbtw: 0.30000000000000004 7.848615102533714
Evaluation of distbtw: 0.3500000000000001 4.637787337213343
Evaluation of distbtw: 0.40000000000000013 3.7566754835725007
Evaluation of distbtw: 0.45000000000000007 2.8739483190429453
Evaluation of distbtw: 0.5000000000000001 1.9899857759802337
Evaluation of distbtw: 0.5500000000000002 1.4001793591839942
Evaluation of distbtw: 0.6000000000000002 0.5150059280635517
Evaluation of distbtw: 0.6500000000000001 8.138540603152052
Evaluation of distbtw: 0.7000000000000002 7.267655974695472
Evaluation of distbtw: 0.7500000000000002 6.976649836222101
Evaluation of distbtw: 0.8000000000000002 6.393653329172267
Evaluation of distbtw: 0.8500000000000002 5.809436703481538
Evaluation of distbtw: 0.9000000000000002 5